# BÁO CÁO: DỰ ĐOÁN KHÁCH HÀNG RỜI BỎ (CUSTOMER CHURN PREDICTION)

**Mục tiêu:**
Xây dựng và so sánh hiệu năng của các mô hình Machine Learning để dự đoán việc khách hàng ngừng sử dụng dịch vụ (Churn).

**Các mô hình được sử dụng:**
1.  Logistic Regression (Hồi quy Logistic)
2.  Random Forest Classifier (Rừng ngẫu nhiên)
3.  Support Vector Machine (SVM)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Thư viện Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Cấu hình hiển thị
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import warnings
warnings.filterwarnings('ignore')

## 1. Tải và Khám phá dữ liệu
Dữ liệu bao gồm các thông tin về dịch vụ, hợp đồng và phương thức thanh toán của khách hàng.

In [ ]:
# Đọc dữ liệu
try:
    df = pd.read_csv("../data/Customer-Churn.csv")
    print("Kích thước dữ liệu:", df.shape)
    display(df.head())
except FileNotFoundError:
    print("Không tìm thấy file dữ liệu!")

# Kiểm tra kiểu dữ liệu và giá trị thiếu
print("\n--- Thông tin dữ liệu ---")
print(df.info())

print("\n--- Số lượng giá trị thiếu (Null) ---")
print(df.isnull().sum())

## 2. Lựa chọn đặc trưng và Gán nhãn
* **Target:** Cột `Churn` dạng "Yes"/"No" được chuyển về dạng số `1`/`0`.
* **Features:** Lựa chọn các đặc trưng quan trọng như: Thời gian sử dụng (`tenure`), Phí hàng tháng (`MonthlyCharges`), Loại hợp đồng (`Contract`), v.v.

In [ ]:
# Chuyển đổi nhãn Target
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})

# Lựa chọn đặc trưng đầu vào (X) và nhãn (y)
features = ["tenure", "MonthlyCharges", "Contract", "InternetService", "PaymentMethod"]
target = "Churn"

X = df[features]
y = df[target]

print("Danh sách đặc trưng (X):", X.columns.tolist())
display(X.head())

## 3. Chia tập dữ liệu Train/Test
Sử dụng tỉ lệ **80% Train - 20% Test**.
* **Lưu ý:** Sử dụng tham số `stratify=y` để đảm bảo tỉ lệ khách hàng rời bỏ (Churn) ở tập Train và Test là tương đương nhau, tránh hiện tượng lệch dữ liệu.



[Image of train test split diagram]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print(f"Kích thước tập Train: {X_train.shape}")
print(f"Kích thước tập Test : {X_test.shape}")

# Kiểm tra tỉ lệ Churn
print("\n--- Tỉ lệ Churn (Mean) ---")
print(f"Toàn bộ : {y.mean():.3f}")
print(f"Train   : {y_train.mean():.3f}")
print(f"Test    : {y_test.mean():.3f}")

## 4. Xây dựng Pipeline xử lý đặc trưng (Feature Engineering)
Chúng ta sử dụng `ColumnTransformer` để áp dụng các kỹ thuật xử lý khác nhau cho từng loại dữ liệu:
* **Numeric Features** (`tenure`, `MonthlyCharges`): Áp dụng **StandardScaler** để đưa về phân phối chuẩn (mean=0, std=1).
* **Categorical Features** (`Contract`, `InternetService`, ...): Áp dụng **OneHotEncoder** để chuyển thành các vector số học.



[Image of data pipeline architecture]

In [ ]:
numeric_features = ["tenure", "MonthlyCharges"]
categorical_features = ["Contract", "InternetService", "PaymentMethod"]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), categorical_features)
    ],
    remainder='passthrough'
)

print("Đã thiết lập bộ tiền xử lý (Preprocessor).")

## 5. Huấn luyện và Đánh giá
Chúng ta sẽ huấn luyện 3 mô hình thông qua Pipeline.
Do dữ liệu Churn thường mất cân bằng, ta sử dụng tham số `class_weight='balanced'` cho các mô hình.

**Các chỉ số đánh giá:**
* **Accuracy:** Độ chính xác tổng thể.
* **Precision:** Trong số những người máy dự đoán sẽ rời bỏ, bao nhiêu người thực sự rời bỏ?
* **Recall:** Máy phát hiện được bao nhiêu phần trăm người thực sự rời bỏ?
* **F1-Score:** Trung bình điều hòa giữa Precision và Recall (Quan trọng nhất khi dữ liệu mất cân bằng).

In [ ]:
# Định nghĩa các Pipeline mô hình
pipelines = {
    "Logistic Regression": Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression(max_iter=1000, class_weight="balanced"))
    ]),
    "Random Forest": Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", RandomForestClassifier(n_estimators=300, random_state=42, class_weight="balanced_subsample"))
    ]),
    "SVM": Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", SVC(kernel="rbf", C=1.0, gamma="scale", class_weight="balanced"))
    ])
}

results = []

print("Bắt đầu huấn luyện...")
for name, model in pipelines.items():
    # Huấn luyện
    model.fit(X_train, y_train)

    # Dự đoán
    y_pred = model.predict(X_test)

    # Tính toán metrics
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1": f1_score(y_test, y_pred)
    })
    print(f"-> Đã xong mô hình {name}")

# Tạo DataFrame kết quả
results_df = pd.DataFrame(results).sort_values(by="F1", ascending=False)

## 6. Tổng kết và So sánh
Bảng dưới đây so sánh hiệu năng của các mô hình, sắp xếp theo F1-Score giảm dần.

In [ ]:
# Hiển thị bảng kết quả
display(results_df)

# Vẽ biểu đồ so sánh F1-Score
plt.figure(figsize=(10, 5))
sns.barplot(x="F1", y="Model", data=results_df, palette="viridis")
plt.title("So sánh F1-Score giữa các mô hình")
plt.xlabel("F1-Score")
plt.xlim(0, 1)
for index, value in enumerate(results_df["F1"]):
    plt.text(value, index, f'{value:.3f}', va='center')
plt.show()